In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Objective is binary classification given the set of features

# Libraries

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
plt.figure(figsize = (20, 18))
import seaborn as sns
from yellowbrick.target import FeatureCorrelation
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
from sklearn.feature_selection import SelectKBest
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report,plot_confusion_matrix,accuracy_score,roc_auc_score,roc_curve,auc
from sklearn.model_selection import train_test_split,StratifiedKFold,cross_val_score
from bayes_opt import BayesianOptimization
import xgboost as xgb
# hyperopt is hyperparameter optimization by defining an objective function and declaring a search space
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from lightgbm import LGBMClassifier
import lightgbm as lgb
from xgboost import XGBClassifier
from cuml import RandomForestClassifier as cuRF
from cuml.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
import xgboost

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Data 

In [ ]:
train_data = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/train.csv")
train_data.shape

In [ ]:
train_data.head()

In [ ]:
train_data.dtypes

In [ ]:
train_data.info() # all features are continuous

In [ ]:
sum(train_data.isnull().sum()) # no null values

In [ ]:
# dropping the id column
train_data = train_data.drop("id", axis=1)

In [ ]:
train_data.head()

In [ ]:
train_data.describe().T # the features are on varying scale

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
# Compresses the training data 
train_data = reduce_mem_usage(train_data)

# EDA

In [ ]:
train_data.f0.describe()

In [ ]:
sns.distplot(train_data['f0'])


In [ ]:
from seaborn import boxplot
fig, ax = plt.subplots(figsize=(8, 6))
boxplot(x="target", y="f0", data=train_data)


In [ ]:
from seaborn import violinplot
fig, ax = plt.subplots(figsize=(8, 6))
violinplot(x="target", y="f0", data=train_data)


In [ ]:
violinplot(x="target", y="f1", data=train_data)

In [ ]:
fig = sns.FacetGrid(train_data, hue="target", aspect=.75, height=4)
fig.map_dataframe(sns.scatterplot, x="f0", y="f1")  # bill_depth_mm vs body_mass_g in the 3 islands and the 3 species
fig.set_axis_labels("f0", "f1")
fig.add_legend()

In [ ]:
violinplot(x="target", y="f2", data=train_data)

In [ ]:
violinplot(x="target", y="f3", data=train_data)

In [ ]:
violinplot(x="target", y="f4", data=train_data)

In [ ]:
violinplot(x="target", y="f5", data=train_data)

In [ ]:
violinplot(x="target", y="f6", data=train_data)

In [ ]:
violinplot(x="target", y="f7", data=train_data)

In [ ]:
violinplot(x="target", y="f8", data=train_data)

In [ ]:
violinplot(x="target", y="f9", data=train_data)

In [ ]:
violinplot(x="target", y="f10", data=train_data)

In [ ]:
fig = sns.FacetGrid(train_data, hue="target", aspect=.75, height=4)
fig.map_dataframe(sns.scatterplot, x="f1", y="f2")  
fig.set_axis_labels("f1", "f2")
fig.add_legend()

In [ ]:
fig = sns.FacetGrid(train_data, hue="target", aspect=.75, height=4)
fig.map_dataframe(sns.scatterplot, x="f0", y="f2")  
fig.set_axis_labels("f0", "f2")
fig.add_legend()

In [ ]:
fig = sns.FacetGrid(train_data, hue="target", aspect=.75, height=4)
fig.map_dataframe(sns.scatterplot, x="f2", y="f3")  
fig.set_axis_labels("f2", "f3")
fig.add_legend()

In [ ]:
fig = sns.FacetGrid(train_data, hue="target", aspect=.75, height=4)
fig.map_dataframe(sns.scatterplot, x="f3", y="f4")  
fig.set_axis_labels("f3", "f4")
fig.add_legend()

In [ ]:
fig = sns.FacetGrid(train_data, hue="target", aspect=.75, height=4)
fig.map_dataframe(sns.scatterplot, x="f4", y="f5")  
fig.set_axis_labels("f4", "f5")
fig.add_legend()

In [ ]:
fig = sns.FacetGrid(train_data, hue="target", aspect=.75, height=4)
fig.map_dataframe(sns.scatterplot, x="f5", y="f6")  
fig.set_axis_labels("f5", "f6")
fig.add_legend()

In [ ]:
fig = sns.FacetGrid(train_data, hue="target", aspect=.75, height=4)
fig.map_dataframe(sns.scatterplot, x="f7", y="f8")  
fig.set_axis_labels("f7", "f8")
fig.add_legend()

In [ ]:
fig = sns.FacetGrid(train_data, hue="target", aspect=.75, height=4)
fig.map_dataframe(sns.scatterplot, x="f9", y="f10")  
fig.set_axis_labels("f9", "f10")
fig.add_legend()

In [ ]:
plt.figure(figsize=(16, 6))
heatmap = sns.heatmap(train_data[['f0','f1','f2','f3','f4','f5','f6','f7','f8','f9','f10']].corr(), vmin=-1, vmax=1, annot=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);

In [ ]:
sns.countplot(data=train_data,x='target')

# Feature Selection with YellowBricks, SelectKBest and Sklearn Mutual Information, Recursive Feature Elimination

#### Ten columns at a time subsetting and applying yellowbricks package to get the important features

In [ ]:
df = train_data[['f0','f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9','f10','target']]

In [ ]:
df.plot.scatter(x='f1', y='f10',c='target',colormap='viridis',figsize=(20, 18));

In [ ]:
target = df['target']

features = df.drop('target', axis=1)

In [ ]:
feature_names = list(features.columns)
figure(figsize=(20,18), dpi=80)
visualizer = FeatureCorrelation(labels = feature_names, method='pearson')

visualizer.fit(features, target)

visualizer.poof()

#### f3, f4, f8, f9, f10

#### With SelectKBest

In [ ]:
select_univariate = SelectKBest(f_classif, k=4).fit(features, target)

features_mask = select_univariate.get_support()

features_mask

selected_columns = features.columns[features_mask]

selected_columns

selected_features = features[selected_columns]

selected_features.head()

#### f3,f4,f8,f10

In [ ]:
from sklearn import feature_selection
mic = feature_selection.mutual_info_classif( features, target)
fig, ax = plt.subplots(figsize=(10, 8))
(pd.DataFrame({"feature": features.columns, "vimp": mic}).set_index("feature").plot.barh(ax=ax))


In [ ]:
from sklearn.feature_selection import RFE
model = RandomForestClassifier( n_estimators=10)
rfe = RFE(model, 4)
rfe.fit(features, target)
print(rfe.support_)
print(features.loc[:, rfe.support_].head())

#### f11 to f20 features

In [ ]:
fig = sns.FacetGrid(train_data, hue="target", aspect=.75, height=4)
fig.map_dataframe(sns.scatterplot, x="f19", y="f20")  # bill_depth_mm vs body_mass_g in the 3 islands and the 3 species
fig.set_axis_labels("f19", "f20")
fig.add_legend()

In [ ]:
df = train_data[['f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19','f20','target']]

In [ ]:
target = df['target']

features = df.drop('target', axis=1)

In [ ]:
feature_names = list(features.columns)
figure(figsize=(20,18), dpi=80)
visualizer = FeatureCorrelation(labels = feature_names, method='pearson')

visualizer.fit(features, target)

visualizer.poof()

#### f14,f16,f17, f20

In [ ]:
select_univariate = SelectKBest(f_classif, k=4).fit(features, target)

features_mask = select_univariate.get_support()

features_mask

selected_columns = features.columns[features_mask]

selected_columns

selected_features = features[selected_columns]

selected_features.head()

#### f14,f16,f17,f20

In [ ]:
mic = feature_selection.mutual_info_classif( features, target)
fig, ax = plt.subplots(figsize=(10, 8))
(pd.DataFrame({"feature": features.columns, "vimp": mic}).set_index("feature").plot.barh(ax=ax))


In [ ]:
from sklearn.feature_selection import RFE
model = RandomForestClassifier( n_estimators=10)
rfe = RFE(model, 4)
rfe.fit(features, target)
print(rfe.support_)
print(features.loc[:, rfe.support_].head())

#### f21 to f30 features

In [ ]:
df = train_data[['f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29','f30','target']]

In [ ]:
target = df['target']

features = df.drop('target', axis=1)

In [ ]:
feature_names = list(features.columns)
figure(figsize=(20,18), dpi=80)
visualizer = FeatureCorrelation(labels = feature_names, method='pearson')

visualizer.fit(features, target)

visualizer.poof()

####  f21, f22, f24, f25, f26, f27, f30 are the features to consider

In [ ]:
select_univariate = SelectKBest(f_classif, k=4).fit(features, target)

features_mask = select_univariate.get_support()

features_mask

selected_columns = features.columns[features_mask]

selected_columns

selected_features = features[selected_columns]

selected_features.head()

In [ ]:
mic = feature_selection.mutual_info_classif( features, target)
fig, ax = plt.subplots(figsize=(10, 8))
(pd.DataFrame({"feature": features.columns, "vimp": mic}).set_index("feature").plot.barh(ax=ax))

In [ ]:
from sklearn.feature_selection import RFE
model = RandomForestClassifier( n_estimators=10)
rfe = RFE(model, 4)
rfe.fit(features, target)
print(rfe.support_)
print(features.loc[:, rfe.support_].head())

#### we will work with features f21, f22, f25, f27

#### f31 to f40 features

In [ ]:
df = train_data[['f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39','f40','target']]

In [ ]:
target = df['target']
features = df.drop('target', axis=1)

In [ ]:
feature_names = list(features.columns)
figure(figsize=(20,18), dpi=80)
visualizer = FeatureCorrelation(labels = feature_names, method='pearson')

visualizer.fit(features, target)

visualizer.poof()

#### f31,f34,f40

In [ ]:
select_univariate = SelectKBest(f_classif, k=4).fit(features, target)

features_mask = select_univariate.get_support()

features_mask

selected_columns = features.columns[features_mask]

selected_columns

selected_features = features[selected_columns]

selected_features.head()

In [ ]:
mic = feature_selection.mutual_info_classif( features, target)
fig, ax = plt.subplots(figsize=(10, 8))
(pd.DataFrame({"feature": features.columns, "vimp": mic}).set_index("feature").plot.barh(ax=ax))

In [ ]:
from sklearn.feature_selection import RFE
model = RandomForestClassifier( n_estimators=10)
rfe = RFE(model, 4)
rfe.fit(features, target)
print(rfe.support_)
print(features.loc[:, rfe.support_].head())

#### will consider f31, f32, f34, f40

#### f41 to f50 features

In [ ]:
df = train_data[['f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49','f50','target']]

In [ ]:
target = df['target']
features = df.drop('target', axis=1)

In [ ]:
feature_names = list(features.columns)
figure(figsize=(20,18), dpi=80)
visualizer = FeatureCorrelation(labels = feature_names, method='pearson')

visualizer.fit(features, target)

visualizer.poof()

#### f41, f43, f44,f47,f49,f50

In [ ]:
select_univariate = SelectKBest(f_classif, k=4).fit(features, target)

features_mask = select_univariate.get_support()

features_mask

selected_columns = features.columns[features_mask]

selected_columns

selected_features = features[selected_columns]

selected_features.head()

In [ ]:
mic = feature_selection.mutual_info_classif( features, target)
fig, ax = plt.subplots(figsize=(10, 8))
(pd.DataFrame({"feature": features.columns, "vimp": mic}).set_index("feature").plot.barh(ax=ax))

In [ ]:
from sklearn.feature_selection import RFE
model = RandomForestClassifier( n_estimators=10)
rfe = RFE(model, 4)
rfe.fit(features, target)
print(rfe.support_)
print(features.loc[:, rfe.support_].head())

#### f41,f43,f44,f50 features

#### f51 to f60

In [ ]:
df = train_data[['f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59','f60','target']]

In [ ]:
target = df['target']
features = df.drop('target', axis=1)

In [ ]:
feature_names = list(features.columns)
figure(figsize=(20,18), dpi=80)
visualizer = FeatureCorrelation(labels = feature_names, method='pearson')

visualizer.fit(features, target)

visualizer.poof()

In [ ]:
select_univariate = SelectKBest(f_classif, k=4).fit(features, target)

features_mask = select_univariate.get_support()

features_mask

selected_columns = features.columns[features_mask]

selected_columns

selected_features = features[selected_columns]

selected_features.head()

In [ ]:
mic = feature_selection.mutual_info_classif( features, target)
fig, ax = plt.subplots(figsize=(10, 8))
(pd.DataFrame({"feature": features.columns, "vimp": mic}).set_index("feature").plot.barh(ax=ax))

In [ ]:
from sklearn.feature_selection import RFE
model = RandomForestClassifier( n_estimators=10)
rfe = RFE(model, 4)
rfe.fit(features, target)
print(rfe.support_)
print(features.loc[:, rfe.support_].head())

#### f54, f55, f57, f60

#### f61 to f70

In [ ]:
df = train_data[['f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69','f70','target']]

In [ ]:
target = df['target']
features = df.drop('target', axis=1)

In [ ]:
feature_names = list(features.columns)
figure(figsize=(20,18), dpi=80)
visualizer = FeatureCorrelation(labels = feature_names, method='pearson')

visualizer.fit(features, target)

visualizer.poof()

In [ ]:
select_univariate = SelectKBest(f_classif, k=4).fit(features, target)

features_mask = select_univariate.get_support()

features_mask

selected_columns = features.columns[features_mask]

selected_columns

selected_features = features[selected_columns]

selected_features.head()

In [ ]:
mic = feature_selection.mutual_info_classif( features, target)
fig, ax = plt.subplots(figsize=(10, 8))
(pd.DataFrame({"feature": features.columns, "vimp": mic}).set_index("feature").plot.barh(ax=ax))

In [ ]:
from sklearn.feature_selection import RFE
model = RandomForestClassifier( n_estimators=10)
rfe = RFE(model, 4)
rfe.fit(features, target)
print(rfe.support_)
print(features.loc[:, rfe.support_].head())

#### f62, f64, f66, f67

#### f71 to f80

In [ ]:
df = train_data[['f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79','f80','target']]

In [ ]:
target = df['target']
features = df.drop('target', axis=1)

In [ ]:
feature_names = list(features.columns)
figure(figsize=(20,18), dpi=80)
visualizer = FeatureCorrelation(labels = feature_names, method='pearson')

visualizer.fit(features, target)

visualizer.poof()

In [ ]:
select_univariate = SelectKBest(f_classif, k=4).fit(features, target)

features_mask = select_univariate.get_support()

features_mask

selected_columns = features.columns[features_mask]

selected_columns

selected_features = features[selected_columns]

selected_features.head()

In [ ]:
mic = feature_selection.mutual_info_classif( features, target)
fig, ax = plt.subplots(figsize=(10, 8))
(pd.DataFrame({"feature": features.columns, "vimp": mic}).set_index("feature").plot.barh(ax=ax))

In [ ]:
from sklearn.feature_selection import RFE
model = RandomForestClassifier( n_estimators=10)
rfe = RFE(model, 4)
rfe.fit(features, target)
print(rfe.support_)
print(features.loc[:, rfe.support_].head())

#### f71, f75, f77, f80

#### f81 to f90

In [ ]:
df = train_data[['f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89','f90','target']]

In [ ]:
target = df['target']
features = df.drop('target', axis=1)

In [ ]:
feature_names = list(features.columns)
figure(figsize=(20,18), dpi=80)
visualizer = FeatureCorrelation(labels = feature_names, method='pearson')

visualizer.fit(features, target)

visualizer.poof()

In [ ]:
select_univariate = SelectKBest(f_classif, k=4).fit(features, target)

features_mask = select_univariate.get_support()

features_mask

selected_columns = features.columns[features_mask]

selected_columns

selected_features = features[selected_columns]

selected_features.head()

In [ ]:
mic = feature_selection.mutual_info_classif( features, target)
fig, ax = plt.subplots(figsize=(10, 8))
(pd.DataFrame({"feature": features.columns, "vimp": mic}).set_index("feature").plot.barh(ax=ax))

In [ ]:
from sklearn.feature_selection import RFE
model = RandomForestClassifier( n_estimators=10)
rfe = RFE(model, 4)
rfe.fit(features, target)
print(rfe.support_)
print(features.loc[:, rfe.support_].head())

#### f81, f82, f83, f90

#### f91 to f100

In [ ]:
df = train_data[['f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99','target']]

In [ ]:
target = df['target']
features = df.drop('target', axis=1)

In [ ]:
feature_names = list(features.columns)
figure(figsize=(20,18), dpi=80)
visualizer = FeatureCorrelation(labels = feature_names, method='pearson')

visualizer.fit(features, target)

visualizer.poof()

In [ ]:
select_univariate = SelectKBest(f_classif, k=4).fit(features, target)

features_mask = select_univariate.get_support()

features_mask

selected_columns = features.columns[features_mask]

selected_columns

selected_features = features[selected_columns]

selected_features.head()

In [ ]:
mic = feature_selection.mutual_info_classif( features, target)
fig, ax = plt.subplots(figsize=(10, 8))
(pd.DataFrame({"feature": features.columns, "vimp": mic}).set_index("feature").plot.barh(ax=ax))

In [ ]:
from sklearn.feature_selection import RFE
model = RandomForestClassifier( n_estimators=10)
rfe = RFE(model, 4)
rfe.fit(features, target)
print(rfe.support_)
print(features.loc[:, rfe.support_].head())

#### f91, f96, f97, f98

In [ ]:
del features,target

# Normalization

In [ ]:
std = preprocessing.StandardScaler()

In [ ]:
std = preprocessing.StandardScaler()
train_data[['f3','f4','f8','f10','f14','f16','f17','f20','f21','f22','f25','f27','f31','f32','f34','f40','f41','f43','f44','f50','f54','f55','f57','f60','f62', 'f64', 'f66', 'f67','f71', 'f75', 'f77', 'f80','f81', 'f82', 'f83', 'f90','f91', 'f96', 'f97', 'f98']] = std.fit_transform(train_data[['f3','f4','f8','f10','f14','f16','f17','f20','f21','f22','f25','f27','f31','f32','f34','f40','f41','f43','f44','f50','f54','f55','f57','f60','f62','f64', 'f66', 'f67','f71', 'f75', 'f77', 'f80','f81', 'f82', 'f83', 'f90','f91', 'f96', 'f97', 'f98']])

In [ ]:
train_data[['f3','f4','f8','f10','f14','f16','f17','f20','f21','f22','f25','f27','f31','f32','f34','f40','f41','f43','f44','f50','f54','f55','f57','f60','f62','f64', 'f66', 'f67','f71', 'f75', 'f77', 'f80','f81', 'f82', 'f83', 'f90','f91', 'f96', 'f97', 'f98']].head()

In [ ]:
y = train_data[['target']]
X = train_data[['f3','f4','f8','f10','f14','f16','f17','f20','f21','f22','f25','f27','f31','f32','f34','f40','f41','f43','f44','f50','f54','f55','f57','f60','f62','f64', 'f66', 'f67','f71', 'f75', 'f77', 'f80','f81', 'f82', 'f83', 'f90','f91', 'f96', 'f97', 'f98']]
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y, test_size=0.3, random_state=42)


# Logistic Regression

In [ ]:
lr = LogisticRegression(random_state=42)


In [ ]:
lr.fit(X_train, y_train)


In [ ]:
lr.score(X_test, y_test)


In [ ]:
y_pred = lr.predict(X_test)

In [ ]:
y_pred

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
from yellowbrick.features import FeatureImportances
fig, ax = plt.subplots(figsize=(16, 14))
fi_viz = FeatureImportances(lr)
fi_viz.fit(X, y)
fi_viz.poof()


#### Stratified cross validation

In [ ]:
X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])



In [ ]:
for model in [LogisticRegression]:
     skflr = model()
     skf = StratifiedKFold(n_splits=10, random_state=42)
     s = cross_val_score(skflr, X, y, scoring="roc_auc", cv=skf)
     print("Accuracy = ", s.mean())


In [ ]:
# Loads test data set
test = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/test.csv")

# Removes ID column as it is not required for prediction
test.drop(["id"], axis=1, inplace=True)

In [ ]:
# Loads submission data set that acts just as a template for submission
submission = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/sample_submission.csv")

In [ ]:
test=test[['f3','f4','f8','f10','f14','f16','f17','f20','f21','f22','f25','f27','f31','f32','f34','f40','f41',
           'f43','f44','f50','f54','f55','f57','f60','f62','f64', 'f66', 'f67','f71', 'f75', 'f77', 'f80','f81', 'f82', 'f83', 'f90','f91', 'f96', 'f97', 'f98']]

In [ ]:
test[['f3','f4','f8','f10','f14','f16','f17','f20','f21','f22','f25','f27','f31','f32','f34','f40','f41',
      'f43','f44','f50','f54','f55','f57','f60','f62','f64', 'f66', 'f67','f71', 'f75', 'f77', 'f80','f81', 'f82', 
      'f83', 'f90','f91', 'f96', 'f97', 'f98']]= pd.DataFrame(preprocessing.scale(test[['f3','f4','f8','f10','f14','f16','f17','f20','f21','f22','f25','f27','f31','f32','f34','f40','f41',
           'f43','f44','f50','f54','f55','f57','f60','f62','f64', 'f66', 'f67','f71', 'f75', 'f77', 'f80','f81', 'f82', 'f83', 'f90','f91', 'f96', 'f97', 'f98']]))

In [ ]:
test[['f3','f4','f8','f10','f14','f16','f17','f20','f21','f22','f25','f27','f31','f32','f34','f40','f41',
           'f43','f44','f50','f54','f55','f57','f60','f62','f64', 'f66', 'f67','f71', 'f75', 'f77', 'f80','f81', 'f82', 'f83', 'f90','f91', 'f96', 'f97', 'f98']].head()

In [ ]:
predictions = lr.predict(test)

In [ ]:
submission["target"] = predictions

In [ ]:
submission

In [ ]:
submission.to_csv("./submission.csv", index=False) ## 0.69399

# Naive Bayes

In [ ]:
nb = GaussianNB()
nb.fit(X_train,y_train)

In [ ]:
nb.score(X_test,y_test)

In [ ]:
predictions = nb.predict(test)

In [ ]:
submission["target"] = predictions

In [ ]:
submission

In [ ]:
submission.to_csv("./submission.csv", index=False) ## 0.63862

# SVM

In [ ]:
svc = SVC(kernel='rbf', C=10, gamma=1, cache_size=2000)

In [ ]:
svc.fit(X_train, y_train)

In [ ]:
y_pred = svc.predict(X_test)

In [ ]:
y_pred=y_pred.reshape(180000,1)

In [ ]:
y_test.shape

In [ ]:
y_pred.shape

In [ ]:
np.sum(y_pred==y_test) / y_test.shape[0] * 100

# RF

In [ ]:
# cuml Random Forest params     
cu_rf_params = { 'n_estimators': 500, 'max_depth':8 } 

#### Convert the datatypes to np.float32 because some cuML models require the input to be np.float32.

In [ ]:
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)

In [ ]:
cu_rf = cuRF(**cu_rf_params)
cu_rf.fit(X_train, y_train)  

In [ ]:
y_pred = cu_rf.predict(X_test)
print(accuracy_score(y_test, y_pred))

In [ ]:
# Create StratifiedKFold object.
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state= 40)
val_acc = []
test_predictions = []
submission_predictions = []

In [ ]:
for fold, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
    x_train_fold, x_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]
    print('Fold', fold )
    
    cu_rf.fit(x_train_fold, y_train_fold)
    print("score : ",cu_rf.score(x_train_fold, y_train_fold))
    
    y_pred = cu_rf.predict(x_test_fold)
    print(accuracy_score(y_test_fold, y_pred))
    
    preds = cu_rf.predict(X_test)
    test_predictions.append(preds)
    
    submission = cu_rf.predict(test)
    submission_predictions.append(submission)

In [ ]:
y_pred = np.mean(np.column_stack(test_predictions), axis=1)
y_pred = y_pred.astype('int32')

In [ ]:
y_test = y_test.astype('int32')

In [ ]:
print(accuracy_score(y_test, y_pred))

In [ ]:
submission_preds = np.mean(np.column_stack(submission_predictions), axis=1)
submission_preds = submission_preds.astype('int32')

In [ ]:
#submission["target"] = submission_preds

In [ ]:
#submission

In [ ]:
#submission['target'].value_counts()

In [ ]:
#submission.to_csv("./submission.csv", index=False) ## 0.64382

# XGBoost

#### StratifiedKFold Cross Validation

In [ ]:
# Create StratifiedKFold object.
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state= 40)

In [ ]:
# Performs cross validation on XGB Classifier

model = XGBClassifier(n_estimators=500,objective='binary:logistic', eval_metric='auc',tree_method='gpu_hist')
model_score = cross_val_score(model, X, y, scoring='roc_auc', cv=skf.split(X, y), n_jobs=-1, verbose=10)

In [ ]:
print(model_score.mean())

In [ ]:
del model_score, model

# Hyperparameter Tuning with Bayesian Optimization

#### a) bayes_opt

In [ ]:
parameter_space = {
    'learning_rate': (0.01, 1.0),
    'n_estimators': (100, 1000),
    'max_depth': (2,10),
    'subsample': (0.4, 1.0),
    'colsample_bytree' :(0.4, 1.0),
    'gamma': (0, 5)}

def xgboost_hyper_param(learning_rate,
                        n_estimators,
                        max_depth,
                        subsample,
                        colsample_bytree,
                        gamma):

    max_depth = int(max_depth)
    n_estimators = int(n_estimators)

    clf = XGBClassifier(
        tree_method='gpu_hist',
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        objective = 'binary:logistic',
        eval_metric='auc',
        gamma=gamma)
    return np.mean(cross_val_score(clf, X, y, cv=5, scoring='roc_auc'))

optimizer = BayesianOptimization(
    f=xgboost_hyper_param,
    pbounds=parameter_space,
    random_state=100,
)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
optimizer.maximize(init_points=2, n_iter=5, acq='ei', xi=0.0)

In [ ]:
optimizer.res

In [ ]:
params_gbm = optimizer.max['params']
params_gbm['max_depth'] = round(params_gbm['max_depth'])
params_gbm['n_estimators'] = round(params_gbm['n_estimators'])
params_gbm

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y, test_size=0.3, random_state=42)

In [ ]:
dtrain = xgb.DMatrix(data=X_train, label=y_train)
dval = xgb.DMatrix(data=X_test, label=y_test)
del X_train,y_train,X_test,y_test
params = {'colsample_bytree': 0.7668000197236247,
 'gamma': 4.791427871088578,
 'learning_rate': 0.015842405367374232,
 'max_depth': 8,
 'n_estimators': 900,
 'subsample': 0.6733049507073745}

params["max_depth"] = int(params["max_depth"])
params["objective"] = "binary:logistic"
params["eval_metric"] = "auc"
params["tree_method"] = "gpu_hist"
    
model = xgb.train(
        params, 
        dtrain, 
        num_boost_round=2000, 
        evals=[(dtrain, 'train'), (dval, 'eval')],
        early_stopping_rounds=50, verbose_eval=200)

In [ ]:
dtest = xgb.DMatrix(data=test)
predictions = model.predict(dtest)

In [ ]:
#submission["target"] = predictions

In [ ]:
#submission

In [ ]:
# Saves test predictions
#submission.to_csv("./submission.csv", index=False) # 0.72384

In [ ]:
del model, predictions

# b) Hyperopt

In [ ]:
fold_no = 1
for train_index, test_index in skf.split(X, y):
    print('Fold = ',fold_no)
    y_val = y.iloc[test_index]
    dtrain = xgb.DMatrix(data=X.iloc[train_index], label=y.iloc[train_index])
    dval = xgb.DMatrix(data=X.iloc[test_index], label=y.iloc[test_index])
    fold_no +=1

In [ ]:
hyperparameter_space = { 
                        'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
                        'max_depth': hp.quniform("max_depth", 2, 6, 1),
                        'min_child_weight' : hp.quniform('min_child_weight', 1, 8, 1),
                        'reg_alpha' : hp.uniform('reg_alpha', 1e-8, 100),
                        'reg_lambda' : hp.uniform('reg_lambda', 1e-8, 100),
                        'gamma': hp.uniform ('gamma', 0.0, 1.0),
                        'subsample': hp.uniform("subsample", 0.1, 1.0),
                        'colsample_bytree': hp.uniform('colsample_bytree', 0.1, 1.0)
                       }

In [ ]:
def optimize_hyppara(hyperparameter_space):
    # Converts parameter value to int as required by XGBoost
    hyperparameter_space["max_depth"] = int(hyperparameter_space["max_depth"])
    hyperparameter_space["objective"] = "binary:logistic"
    hyperparameter_space["eval_metric"] = "auc"
    hyperparameter_space["tree_method"] = "gpu_hist"
    
    model = xgb.train(
        hyperparameter_space, 
        dtrain, 
        num_boost_round=2000, 
        evals=[(dtrain, 'train'), (dval, 'eval')],
        early_stopping_rounds=50, verbose_eval=False)
    
    predictions = model.predict(dval)
    
    roc_auc = roc_auc_score(y_val, predictions)
    
    del predictions, model, hyperparameter_space
    
    return {"loss": -roc_auc, "status": STATUS_OK}

In [ ]:
# Starts hyperparameters tuning
trials = Trials()
best_model_params = fmin(fn=optimize_hyppara,space=hyperparameter_space, max_evals=50,algo=tpe.suggest,trials=trials)

In [ ]:
best_model_params

In [ ]:
del dtrain, dval,y_val

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
dtrain = xgb.DMatrix(data=X_train, label=y_train)
dval = xgb.DMatrix(data=X_test, label=y_test)
del X_train,y_train,X_test,y_test
params = {'colsample_bytree': 0.10515757933101197,
 'gamma': 0.6542994638529418,
 'learning_rate': 0.06786577881330061,
 'max_depth': 2.0,
 'min_child_weight': 6.0,
 'reg_alpha': 27.332999614653477,
 'reg_lambda': 30.889493181700296,
 'subsample': 0.9965789712247324}

params["max_depth"] = int(params["max_depth"])
params["objective"] = "binary:logistic"
params["eval_metric"] = "auc"
params["tree_method"] = "gpu_hist"
    
model = xgb.train(
        params, 
        dtrain, 
        num_boost_round=2000, 
        evals=[(dtrain, 'train'), (dval, 'eval')],
        early_stopping_rounds=50, verbose_eval=200)

In [ ]:
# Adds other important parameters
best_model_params["max_depth"] = int(best_model_params["max_depth"])
best_model_params["objective"] = "binary:logistic"
best_model_params["eval_metric"] = "auc"
best_model_params["tree_method"] = "gpu_hist"

In [ ]:
dtest = xgb.DMatrix(data=test)
predictions = model.predict(dtest)

In [ ]:
#submission["target"] = predictions

# Checks for sumbission file before saving
#submission

In [ ]:
# Saves test predictions
#submission.to_csv("./submission.csv", index=False) # 0.72613

In [ ]:
del model, dtest, predictions

In [ ]:
# Gets the model trained over cross validation and predictions 
# against each iteration is stored

test_predictions = []

dtest = xgb.DMatrix(data=test)

for fold, (train_index, val_index) in enumerate(skf.split(X, y)):
    print("fold", fold)

    dtrain = xgb.DMatrix(data=X.iloc[train_index], label=y.iloc[train_index])
    dval = xgb.DMatrix(data=X.iloc[val_index], label=y.iloc[val_index])
    
    model = xgb.train(
        best_model_params, 
        dtrain, 
        num_boost_round=2000, 
        evals=[(dtrain, 'train'), (dval, 'eval')],
        early_stopping_rounds=50, verbose_eval=200)
    
    predictions = model.predict(dtest)
    
    test_predictions.append(predictions)
    
    del predictions, model, dval, dtrain

In [ ]:
test_predictions

In [ ]:
del dtest

In [ ]:
# Predictions stored against each cross validation iteration finally gets aeveraged
# and target column is set with that averaged predictions
#submission["target"] = np.mean(np.column_stack(test_predictions), axis=1)

# Checks for sumbission file before saving
#submission

In [ ]:
# Saves test predictions
#submission.to_csv("./submission.csv", index=False # 0.72667

# LightGBM and GPU and Tuning

In [ ]:
model = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=- 1, learning_rate=0.1, n_estimators=300, device = "gpu")

In [ ]:
# Performs cross validation on LGBM Classifier
model_score = cross_val_score(model, X, y, scoring='roc_auc', cv=skf.split(X, y), n_jobs=-1, verbose=10)

In [ ]:
print(model_score.mean())

In [ ]:
del model, model_score

In [ ]:
parameter_space = {
    'learning_rate': (0.01, 1.0),
    'n_estimators': (100, 1000),
    'num_leaves': (80, 100),
    'feature_fraction': (0.1, 0.9),
    'bagging_fraction': (0.8, 1),
     'max_depth': (17, 25),
     'min_split_gain': (0.001, 0.1),
     'min_child_weight': (10, 25),
     'colsample_bytree' :(0.4, 1.0)
   }



In [ ]:
def lgb_hyper_param(learning_rate,
                        n_estimators,
                        num_leaves,
                        feature_fraction,
                        bagging_fraction,
                        max_depth,
                        min_split_gain,
                        min_child_weight,
                        colsample_bytree):

    max_depth = int(max_depth)
    n_estimators = int(n_estimators)

    clf = LGBMClassifier(
        device='gpu',
        boosting_type='gbdt',
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        objective = 'binary',
        metric='auc')
    return np.mean(cross_val_score(clf, X, y, cv=5, scoring='roc_auc'))

In [ ]:
optimizer = BayesianOptimization(
    f=lgb_hyper_param,
    pbounds=parameter_space,
    random_state=100,
)

In [ ]:
optimizer.maximize(init_points=2, n_iter=5, acq='ei', xi=0.0)

In [ ]:
optimizer.res

In [ ]:
params_gbm = optimizer.max['params']
params_gbm['max_depth'] = round(params_gbm['max_depth'])
params_gbm['n_estimators'] = round(params_gbm['n_estimators'])
params_gbm

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
dtrain = lgb.Dataset(X_train, label=y_train)
dval = lgb.Dataset(X_test, label=y_test,reference=dtrain)

In [ ]:
del X_train,y_train,X_test,y_test
params ={'bagging_fraction': 0.9428901602120185,
 'colsample_bytree': 0.4155177024177823,
 'feature_fraction': 0.8821389082620034,
 'learning_rate': 0.23807965335569445,
 'max_depth': 19,
 'min_child_weight': 13.197890748572707,
 'min_split_gain': 0.0887958747434185,
 'n_estimators': 415,
 'num_leaves': 82.36988658853868}

params["max_depth"] = int(params["max_depth"])
params["num_leaves"] = int(params["num_leaves"])
params["objective"] = "binary"
params["metric"] = "auc"
params["device"] = "gpu"
    
model = lgb.train(
        params, 
        dtrain, 
        num_boost_round=2000, 
        valid_sets =(dval,),
        early_stopping_rounds=50, verbose_eval=200)

In [ ]:

predictions = model.predict(test)

In [ ]:
predictions

In [ ]:
#submission["target"] = predictions

In [ ]:
#submission

In [ ]:
# Saves test predictions
#submission.to_csv("./submission.csv", index=False) # 0.71

In [ ]:
fold_no = 1
for train_index, test_index in skf.split(X, y):
    print('Fold = ',fold_no)
    y_val = y.iloc[test_index]
    dtrain =  lgb.Dataset(data=X.iloc[train_index], label=y.iloc[train_index])
    dval =  lgb.Dataset(data=X.iloc[test_index], label=y.iloc[test_index])
    fold_no +=1                    

In [ ]:
hyperparameter_space = { 
                        'learning_rate': hp.uniform('learning_rate', 0.01, 1.0),
                        'max_depth': hp.quniform("max_depth", 2, 20,1),
                        'n_estimators': hp.uniform("n_estimators",100,1000),
                        'num_leaves': hp.uniform("num_leaves", 80,100),
                        'feature_fraction': hp.uniform("feature_fraction",0.1, 0.9),
                        'bagging_fraction': hp.uniform("bagging_fraction",0.8, 1),
                        'min_split_gain' : hp.uniform('min_split_gain', 0.001, 0.1),
                        'min_child_weight': hp.quniform('min_child_weight',10, 25,1),
                        'reg_alpha' : hp.uniform('reg_alpha', 1e-8, 100),
                        'reg_lambda' : hp.uniform('reg_lambda', 1e-8, 100)
                       }

In [ ]:
def optimize_hyppara(hyperparameter_space):
    # Converts parameter value to int as required by XGBoost
    hyperparameter_space["max_depth"] = int(hyperparameter_space["max_depth"])
    hyperparameter_space["n_estimators"] = int(hyperparameter_space["n_estimators"])
    hyperparameter_space["num_leaves"] = int(hyperparameter_space["num_leaves"])
    hyperparameter_space["objective"] = "binary"
    hyperparameter_space["metric"] = "auc"
    hyperparameter_space["device"] = "gpu"
    
    model = lgb.train(
        hyperparameter_space, 
        dtrain, 
        num_boost_round=2000, 
        valid_sets=[dval],
        early_stopping_rounds=50, verbose_eval=False,valid_names=['valid'])
   
    score = model.best_score['valid']['auc']
   
    return score
    
    #del predictions, model, hyperparameter_space
    
    #return {"loss": -roc_auc, "status": STATUS_OK}

In [ ]:
# Starts hyperparameters tuning
trials = Trials()
best_model_params = fmin(fn=optimize_hyppara,space=hyperparameter_space, max_evals=50,algo=tpe.suggest,trials=trials)

In [ ]:
best_model_params

In [ ]:
params = {'bagging_fraction': 0.9115199584760522,
 'feature_fraction': 0.8903279132717933,
 'learning_rate': 0.9991958623009387,
 'max_depth': 20.0,
 'min_child_weight': 25.0,
 'min_split_gain': 0.08401863889755357,
 'n_estimators': 747.8900198385907,
 'num_leaves': 99.60204048837596,
 'reg_alpha': 35.87885686359565,
 'reg_lambda': 98.47582584848364}

params["max_depth"] = int(params["max_depth"])
params["n_estimators"] = int(params["n_estimators"])
params["num_leaves"] = int(params["num_leaves"])
params["objective"] = "binary"
params["metric"] = "auc"
params["device"] = "gpu"
    
model = lgb.train(
        params, 
        dtrain, 
        num_boost_round=2000, 
        valid_sets=[dval],
        early_stopping_rounds=100, verbose_eval=200)

In [ ]:
predictions = model.predict(test)

In [ ]:
#submission["target"] = predictions

# Checks for sumbission file before saving
#submission

In [ ]:
# Saves test predictions
#submission.to_csv("./submission.csv", index=False) # 0.69651

In [ ]:
# Gets the model trained over cross validation and predictions 
# against each iteration is stored

test_predictions = []

In [ ]:
best_model_params["max_depth"] = int(best_model_params["max_depth"])
best_model_params["n_estimators"] = int(best_model_params["n_estimators"])
best_model_params["num_leaves"] = int(best_model_params["num_leaves"])
best_model_params["objective"] = "binary"
best_model_params["metric"] = "auc"
best_model_params["device"] = "gpu"

In [ ]:
for fold, (train_index, val_index) in enumerate(skf.split(X, y)):
    print("fold", fold)

    dtrain = lgb.Dataset(data=X.iloc[train_index], label=y.iloc[train_index])
    dval = lgb.Dataset(data=X.iloc[val_index], label=y.iloc[val_index])
    
    model = lgb.train(
        best_model_params, 
        dtrain, 
        num_boost_round=2000, 
        valid_sets=[dval],
        early_stopping_rounds=50, verbose_eval=200)
    
    predictions = model.predict(test)
    
    test_predictions.append(predictions)
    
    del predictions, model, dval, dtrain

In [ ]:
test_predictions

In [ ]:
# Predictions stored against each cross validation iteration finally gets aeveraged
# and target column is set with that averaged predictions
#@submission["target"] = np.mean(np.column_stack(test_predictions), axis=1)

In [ ]:
# Checks for sumbission file before saving
#submission

In [ ]:
# Saves test predictions
#submission.to_csv("./submission.csv", index=False )

#### https://machinelearningmastery.com/blending-ensemble-machine-learning-with-python/

# Blending

In [ ]:
# split dataset into train and test sets
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# split training set into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.33, random_state=1)
# summarize data split
print('Train: %s, Val: %s, Test: %s' % (X_train.shape, X_val.shape, X_test.shape))

In [ ]:
# get a list of base models
def get_models():
    models = list()
    models.append(('lr', LogisticRegression()))
    models.append(('bayes', GaussianNB()))
    return models

In [ ]:
# fit the blending ensemble
def fit_ensemble(models, X_train, X_val, y_train, y_val):
    # fit all models on the training set and predict on hold out set
    meta_X = list()
    for name, model in models:
        # fit in training set
        model.fit(X_train, y_train)
        # predict on hold out set
        yhat = model.predict(X_val)
        # reshape predictions into a matrix with one column
        yhat = yhat.reshape(len(yhat), 1)
        # store predictions as input for blending
        meta_X.append(yhat)
        # create 2d array from predictions, each set is an input feature
        meta_X = np.hstack(meta_X)
        # define blending model
        blender = LogisticRegression()
        # fit on predictions from base models
        blender.fit(meta_X, y_val)
        return blender

In [ ]:
# make a prediction with the blending ensemble
def predict_ensemble(models, blender, X_test):
    # make predictions with base models
    meta_X = list()
    for name, model in models:
        # predict with base model
        yhat = model.predict(X_test)
        # reshape predictions into a matrix with one column
        yhat = yhat.reshape(len(yhat), 1)
        # store prediction
        meta_X.append(yhat)
        # create 2d array from predictions, each set is an input feature
        meta_X = np.hstack(meta_X)
        # predict
        return blender.predict(meta_X)

In [ ]:
...
# create the base models
models = get_models()
# train the blending ensemble
blender = fit_ensemble(models, X_train, X_val, y_train, y_val)
# make predictions on test set
yhat = predict_ensemble(models, blender, X_test)

In [ ]:
...
# evaluate predictions
score = accuracy_score(y_test, yhat)
print('Blending Accuracy: %.3f' % score)

In [ ]:
predictions = predict_ensemble(models, blender, test)

In [ ]:
predictions

In [ ]:
#submission["target"] = predictions
#submission

In [ ]:
# Saves test predictions
#submission.to_csv("./submission.csv", index=False ) #0.69408

# Stacking

In [ ]:
model1 = LogisticRegression(random_state=123)
model1.fit(X_train, y_train)


In [ ]:
model2 = GaussianNB()
model2.fit(X_train,y_train)

In [ ]:
model3 = xgboost.XGBClassifier(
        tree_method='gpu_hist',
        objective = 'binary:logistic',
        eval_metric='auc')
model3.fit(X_train,y_train)

In [ ]:
preds1=model1.predict(X_val)
preds2=model2.predict(X_val)
preds3=model3.predict(X_val)

In [ ]:

test_preds1=model1.predict(X_test)
test_preds2=model2.predict(X_test)
test_preds3=model3.predict(X_test)

In [ ]:
final_preds1=model1.predict(test)
final_preds2=model2.predict(test)
final_preds3=model3.predict(test)

In [ ]:
# form a new dataset for valid and test via stackng the predictions

stacked_predictions = np.column_stack((preds1,preds2,preds3))
stacked_predictions

In [ ]:
stacked_test_preds = np.column_stack((test_preds1,test_preds2,test_preds3))
stacked_test_preds

In [ ]:
final_stacked_test_preds = np.column_stack((final_preds1,final_preds2,final_preds3))
final_stacked_test_preds

In [ ]:
model = LogisticRegression(random_state=123)
model.fit(stacked_predictions,y_val)
model.score(stacked_test_preds,y_test)

In [ ]:
#submission["target"] = final_stacked_test_preds
#submission

In [ ]:
# Saves test predictions
#submission.to_csv("./submission.csv", index=False ) # 0.69408

# Sequential Dense Model (ANN) with Keras and Tensorflow

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:

early_stopping = EarlyStopping(
 min_delta=0.001, # Minimium amount of change to count as an improvement
 patience=10, # How many epochs to wait before stopping
 restore_best_weights=True)

In [ ]:

model = Sequential([
 Dense(80, activation='relu', input_shape=[X_train.shape[1]]),
 Dropout(0.2),
 BatchNormalization(),
 Dense(40, activation='relu'),
 Dropout(0.2),
 BatchNormalization(),
 Dense(20, activation='relu'),
 Dropout(0.2),
 BatchNormalization(),
 Dense(1, activation='sigmoid')])


In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['AUC'])

In [ ]:
BATCH_SIZE=128

In [ ]:
model.fit(
 X_train, y_train,
 validation_data=(X_val, y_val),
 batch_size=BATCH_SIZE,
 epochs=20,
 callbacks=[early_stopping], 
 verbose=1) 

In [ ]:
model.evaluate(X_test, y_test, verbose=0)

In [ ]:
predictions = (model.predict(X_test) > 0.5).astype("int32")

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
preds = (model.predict(test) > 0.5).astype("int32")

In [ ]:
preds

In [ ]:
#submission["target"] = preds

In [ ]:
#submission

In [ ]:
#submission.to_csv("./submission.csv", index=False ) # 0.693